In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os
path = os.path.dirname(os.getcwd())
path = os.path.join(path, "ProcessedData")

In [ ]:
train = pd.read_csv(os.path.join(path, "train_processed.csv"))
test = pd.read_csv(os.path.join(path, "test_processed.csv"))
apt_train = pd.read_csv(os.path.join(path, "building_tr.csv"))
apt_test = pd.read_csv(os.path.join(path, "building_tst.csv"))
adr_train = pd.read_csv(os.path.join(path, "train_apt.csv"))
adr_test = pd.read_csv(os.path.join(path, "test_apt.csv"))
subway = pd.read_csv(os.path.join(path, "subway.csv"))

In [ ]:
apt_train = apt_train.iloc[:,[4,9]].rename(columns={'건물명':'단지명'})
apt_test = apt_test.iloc[:,[4,9]].rename(columns={'건물명':'단지명'})

In [ ]:
train = pd.merge(train,apt_train,how='left',on='도로명주소')
train = pd.merge(train,adr_train.loc[:,['위도','경도','도로명주소']],how='left',on='도로명주소')

In [ ]:
test = pd.merge(test,apt_test,how='left',on='도로명주소')
test = pd.merge(test,adr_test.loc[:,['위도','경도','도로명주소']],how='left',on='도로명주소')

In [ ]:
def loc(x,y):
    m = 1000000; res=''
    for ind in range(subway.shape[0]):
        sub_x = subway.loc[ind,'위도']
        sub_y = subway.loc[ind,'경도']
        subway_name = subway.loc[ind,'subway_name']
        distance = np.sqrt((sub_x-x)**2+(sub_y-y)**2)
        if distance < m:
            m = distance
            res = subway_name
    return res,m

In [ ]:
tmp = train.apply(lambda x : loc(x['위도'],x['경도']),axis=1)
tmp1 = test.apply(lambda x : loc(x['위도'],x['경도']),axis=1)

In [ ]:
train['subway_name'] = [i[0] for i in tmp]
train['subway_dist'] = [i[1] for i in tmp]

In [ ]:
test['subway_name'] = [i[0] for i in tmp1]
test['subway_dist'] = [i[1] for i in tmp1]

In [ ]:
train = pd.merge(train,subway[['subway_name','환승역 수']],on='subway_name',how='left')
test = pd.merge(test,subway[['subway_name','환승역 수']],on='subway_name',how='left')

In [ ]:
import os
path = os.path.dirname(os.getcwd())
path = os.path.join(path, "ProcessedData")
train.to_csv(os.path.join(path,"merged_train.csv"),index=False)
test.to_csv(os.path.join(path,"merged_test.csv"),index=False)